## Search algorithms within Optuna



We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

# Data Pre-processing

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.


In [92]:
data = pd.read_csv('balanced_bank.csv', index_col = 0)
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
34579,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,...,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
446,42,technician,married,professional.course,no,no,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28870,45,blue-collar,married,unknown,no,yes,no,cellular,apr,thu,...,1,999,1,failure,-1.8,93.075,-47.1,1.410,5099.1,yes
30452,60,admin.,married,university.degree,no,yes,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.354,5099.1,yes
28937,38,blue-collar,married,basic.9y,no,yes,no,cellular,apr,fri,...,1,999,1,failure,-1.8,93.075,-47.1,1.405,5099.1,no
16739,41,technician,single,university.degree,no,no,no,cellular,jul,thu,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no


In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9280 entries, 34579 to 16807
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             9280 non-null   int64  
 1   job             9280 non-null   object 
 2   marital         9280 non-null   object 
 3   education       9280 non-null   object 
 4   default         9280 non-null   object 
 5   housing         9280 non-null   object 
 6   loan            9280 non-null   object 
 7   contact         9280 non-null   object 
 8   month           9280 non-null   object 
 9   day_of_week     9280 non-null   object 
 10  duration        9280 non-null   int64  
 11  campaign        9280 non-null   int64  
 12  pdays           9280 non-null   int64  
 13  previous        9280 non-null   int64  
 14  poutcome        9280 non-null   object 
 15  emp.var.rate    9280 non-null   float64
 16  cons.price.idx  9280 non-null   float64
 17  cons.conf.idx   9280 non-nul

In [94]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
34579,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,...,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
446,42,technician,married,professional.course,no,no,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


In [95]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [96]:
data.loan.value_counts()

no         7691
yes        1371
unknown     218
Name: loan, dtype: int64

In [97]:
data.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

# Categorical Encoding

In [98]:
categorical_cols = ['job','marital','education','month','day_of_week'] 
data_encoded = pd.get_dummies(data, drop_first=True,columns = categorical_cols)

# Binary encoding


In [107]:
binary_cat_cols = ['contact','housing','loan','poutcome','default']

for i in binary_cat_cols:
    if len(data_encoded[i].unique())==2:
        data_encoded[i].replace({data_encoded[i].unique()[0]: 0, data_encoded[i].unique()[1]: 1, 'unknown':-1}, inplace=True)
    else:
        data_encoded[i].replace({data_encoded[i].unique()[0]: 0, data_encoded[i].unique()[1]: 1, data_encoded[i].unique()[2]: 2, 'unknown':-1}, inplace=True)


# Label encoding

In [101]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data_encoded['y'] = label_encoder.fit_transform(data_encoded['y'])

In [109]:
data_encoded.dtypes

age                                int64
default                            int64
housing                            int64
loan                               int64
contact                            int64
duration                           int64
campaign                           int64
pdays                              int64
previous                           int64
poutcome                           int64
emp.var.rate                     float64
cons.price.idx                   float64
cons.conf.idx                    float64
euribor3m                        float64
nr.employed                      float64
y                                  int32
job_blue-collar                    uint8
job_entrepreneur                   uint8
job_housemaid                      uint8
job_management                     uint8
job_retired                        uint8
job_self-employed                  uint8
job_services                       uint8
job_student                        uint8
job_technician  

# Scaling

In [114]:
print(data_encoded.shape)

X = data_encoded.loc[:,data_encoded.columns!='y']
print(X.shape)

y = data_encoded.loc[:,'y']
print(y.shape)

(9280, 50)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [120]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Baseline model

In [123]:
model = xgb.XGBClassifier()  
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
accuracy

0.8854064642507345

# HyperParameter optimization with Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [126]:
def objective(trial):
    param = {
        "objective": 'binary:logistic',
        'reg_lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3,1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [i/10.0 for i in range(4,11)]),
        'subsample': trial.suggest_categorical('subsample', [i/10.0 for i in range(4,11)]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.300000012]),
        'n_estimators': trial.suggest_int('n_estimators',100,500),
        'max_depth': trial.suggest_categorical('max_depth', [5,6,7,9,11,13,15,17,20]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)  
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [129]:
random_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)

tpe_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)

cmaes_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

cmaes_study.optimize(objective, n_trials=100)
tpe_study.optimize(objective, n_trials=100)
random_study.optimize(objective, n_trials=100)


[I 2021-12-15 12:12:28,145] A new study created in memory with name: no-name-ceff711a-675b-4d3a-8412-dd41bdd46069
[I 2021-12-15 12:12:28,163] A new study created in memory with name: no-name-6129193f-1413-4936-b941-ab35c1e0205c
[I 2021-12-15 12:12:28,163] A new study created in memory with name: no-name-17091bf6-10bf-45bf-af4b-19fc828f0e83
[I 2021-12-15 12:12:29,713] Trial 0 finished with value: 0.8632060071825008 and parameters: {'alpha': 5.011046519909387, 'gamma': 0.0017533801700933796, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 169, 'max_depth': 5, 'min_child_weight': 146}. Best is trial 0 with value: 0.8632060071825008.
[W 2021-12-15 12:12:29,751] The parameter 'colsample_bytree' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_indepe

[W 2021-12-15 12:12:40,226] The parameter 'max_depth' in trial#4 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-12-15 12:12:43,919] Trial 4 finished with value: 0.8645119164218087 and parameters: {'alpha': 0.16155955553175586, 'gamma': 0.12136883630199864, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.016, 'n_estimators': 299, 'max_depth': 13, 'min_child_weight': 152}. Best is trial 3 with value: 0.8707149853085211.
[W 2021-12-15 12:12:43,919] The parameter 'colsample_bytree' in trial#5 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic

[W 2021-12-15 12:12:53,688] The parameter 'max_depth' in trial#8 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-12-15 12:12:57,073] Trial 8 finished with value: 0.8687561214495593 and parameters: {'alpha': 0.09956573649795873, 'gamma': 0.023543222459501422, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.016, 'n_estimators': 301, 'max_depth': 20, 'min_child_weight': 149}. Best is trial 5 with value: 0.8726738491674828.
[W 2021-12-15 12:12:57,081] The parameter 'colsample_bytree' in trial#9 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynami

[W 2021-12-15 12:13:05,859] The parameter 'max_depth' in trial#12 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-12-15 12:13:09,526] Trial 12 finished with value: 0.8592882794645772 and parameters: {'alpha': 0.004546056243581423, 'gamma': 0.06379460202114186, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.008, 'n_estimators': 301, 'max_depth': 6, 'min_child_weight': 151}. Best is trial 9 with value: 0.880835781913157.
[W 2021-12-15 12:13:09,526] The parameter 'colsample_bytree' in trial#13 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynam

[W 2021-12-15 12:13:19,304] The parameter 'max_depth' in trial#16 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-12-15 12:13:22,483] Trial 16 finished with value: 0.8707149853085211 and parameters: {'alpha': 0.006065588499480185, 'gamma': 0.006893548348122135, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.016, 'n_estimators': 298, 'max_depth': 9, 'min_child_weight': 151}. Best is trial 9 with value: 0.880835781913157.
[W 2021-12-15 12:13:22,483] The parameter 'colsample_bytree' in trial#17 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dyna

...

[I 2021-12-15 12:28:04,319] Trial 85 finished with value: 0.8841005550114267 and parameters: {'alpha': 0.064636369630483, 'gamma': 0.02216405972324701, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.018, 'n_estimators': 256, 'max_depth': 20, 'min_child_weight': 11}. Best is trial 69 with value: 0.8896506692784851.
[I 2021-12-15 12:28:08,765] Trial 86 finished with value: 0.8792033953640223 and parameters: {'alpha': 0.15963556079416108, 'gamma': 0.00851700754975302, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.018, 'n_estimators': 316, 'max_depth': 20, 'min_child_weight': 37}. Best is trial 69 with value: 0.8896506692784851.
[I 2021-12-15 12:28:12,528] Trial 87 finished with value: 0.8821416911524649 and parameters: {'alpha': 0.26641132052669586, 'gamma': 0.014453384962049183, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.018, 'n_estimators': 228, 'max_depth': 20, 'min_child_weight': 21}. Best is trial 69 with value: 0.8896506692784851.
[I

[I 2021-12-15 12:29:47,293] Trial 9 finished with value: 0.8863858961802155 and parameters: {'alpha': 0.026583106345722737, 'gamma': 0.002664239786499342, 'colsample_bytree': 0.4, 'subsample': 0.5, 'learning_rate': 0.300000012, 'n_estimators': 349, 'max_depth': 20, 'min_child_weight': 25}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:29:51,956] Trial 10 finished with value: 0.8537381651975188 and parameters: {'alpha': 0.0012705542554527004, 'gamma': 0.005019003816808144, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.018, 'n_estimators': 466, 'max_depth': 9, 'min_child_weight': 266}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:29:53,068] Trial 11 finished with value: 0.8504733920992491 and parameters: {'alpha': 0.012613419798082668, 'gamma': 0.09125680704982307, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.008, 'n_estimators': 117, 'max_depth': 9, 'min_child_weight': 115}. Best is trial 2 with value: 0.88801828272

[I 2021-12-15 12:31:10,118] Trial 33 finished with value: 0.8684296441397323 and parameters: {'alpha': 0.5138576510755916, 'gamma': 0.0034437857959534715, 'colsample_bytree': 0.6, 'subsample': 1.0, 'learning_rate': 0.02, 'n_estimators': 218, 'max_depth': 6, 'min_child_weight': 242}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:31:14,280] Trial 34 finished with value: 0.871367939928175 and parameters: {'alpha': 0.06874792972889639, 'gamma': 0.448730318634238, 'colsample_bytree': 0.4, 'subsample': 0.5, 'learning_rate': 0.01, 'n_estimators': 480, 'max_depth': 17, 'min_child_weight': 97}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:31:15,607] Trial 35 finished with value: 0.8677766895200784 and parameters: {'alpha': 1.0001816308275862, 'gamma': 0.017890787621499284, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.018, 'n_estimators': 109, 'max_depth': 17, 'min_child_weight': 91}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-

[I 2021-12-15 12:32:27,912] Trial 57 finished with value: 0.8674502122102514 and parameters: {'alpha': 0.02208639830228754, 'gamma': 0.013188438083231729, 'colsample_bytree': 0.4, 'subsample': 0.5, 'learning_rate': 0.018, 'n_estimators': 163, 'max_depth': 13, 'min_child_weight': 107}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:32:32,245] Trial 58 finished with value: 0.8700620306888671 and parameters: {'alpha': 0.08257321830694402, 'gamma': 0.00722725210668693, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.009, 'n_estimators': 374, 'max_depth': 9, 'min_child_weight': 62}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:32:34,058] Trial 59 finished with value: 0.8681031668299053 and parameters: {'alpha': 7.941115497293602, 'gamma': 0.004311461958205857, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.009, 'n_estimators': 166, 'max_depth': 20, 'min_child_weight': 19}. Best is trial 2 with value: 0.8880182827293504.
[I 2

[I 2021-12-15 12:33:55,496] Trial 81 finished with value: 0.8782239634345413 and parameters: {'alpha': 0.0032367818060432953, 'gamma': 0.2284555263706674, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.02, 'n_estimators': 401, 'max_depth': 7, 'min_child_weight': 62}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:33:58,756] Trial 82 finished with value: 0.8684296441397323 and parameters: {'alpha': 0.003043922480118058, 'gamma': 0.30355077048057894, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.300000012, 'n_estimators': 295, 'max_depth': 6, 'min_child_weight': 180}. Best is trial 2 with value: 0.8880182827293504.
[I 2021-12-15 12:34:04,086] Trial 83 finished with value: 0.8677766895200784 and parameters: {'alpha': 0.0039053570257026435, 'gamma': 0.005752106331805448, 'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 0.02, 'n_estimators': 477, 'max_depth': 20, 'min_child_weight': 162}. Best is trial 2 with value: 0.8880182827293

In [144]:
search_space = {
    "n_estimators": [100, 500, 1000],
    "max_depth": [5,6,7,9,11,13,15,17,20],
    "min_samples_split": [0.1, 1.0],
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],

}

def objective(trial):
    
    
    param = {
        'n_estimators': trial.suggest_int('n_estimators',100,500),
        'max_depth': trial.suggest_categorical('max_depth', [5,6,7,9,11,13,15,17,20]),
    }
    model = xgb.XGBClassifier(**param)  
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [145]:
grid_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

grid_study.optimize(objective)

[I 2021-12-15 14:10:36,915] A new study created in memory with name: no-name-ddca2587-a365-414b-b1ff-496d9bae4fab
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:10:52,250] Trial 0 finished with value: 0.8788769180541952 and parameters: {'n_estimators': 1000, 'max_depth': 5}. Best is trial 0 with value: 0.8788769180541952.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "


[I 2021-12-15 14:18:34,195] Trial 22 finished with value: 0.8847535096310807 and parameters: {'n_estimators': 100, 'max_depth': 6}. Best is trial 3 with value: 0.8860594188703885.
[I 2021-12-15 14:18:36,163] Trial 23 finished with value: 0.8860594188703885 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 3 with value: 0.8860594188703885.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:19:17,824] Trial 24 finished with value: 0.881488736532811 and parameters: {'n_estimators': 1000, 'max_depth': 17}. Best is trial 3 with value: 0.8860594188703885.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The valu

[I 2021-12-15 14:29:44,875] Trial 42 finished with value: 0.8788769180541952 and parameters: {'n_estimators': 1000, 'max_depth': 5}. Best is trial 3 with value: 0.8860594188703885.
[I 2021-12-15 14:29:51,110] Trial 43 finished with value: 0.8827946457721189 and parameters: {'n_estimators': 100, 'max_depth': 9}. Best is trial 3 with value: 0.8860594188703885.
[I 2021-12-15 14:30:00,833] Trial 44 finished with value: 0.881162259222984 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 3 with value: 0.8860594188703885.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:30:43,603] Trial 45 finished with value: 0.881162259222984 and parameters: {'n_estimators

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:39:17,118] Trial 68 finished with value: 0.881815213842638 and parameters: {'n_estimators': 1000, 'max_depth': 9}. Best is trial 66 with value: 0.8867123734900424.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:39:35,773] Trial 69 finished with value: 0.881162259222984 and parameters: {'n_estimators': 1000

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 14:46:27,166] Trial 89 finished with value: 0.881162259222984 and parameters: {'n_estimators': 1000, 'max_depth': 15}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 14:46:29,249] Trial 90 finished with value: 0.8827946457721189 and parameters: {'n_estimators': 100, 'max_depth': 7}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 14:46:30,838] Trial 91 finished with value: 0.8860594188703885 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 14:46:32,954] Trial 92 finished with value: 0.8827946457721189 and parameters: {'n_estima

...

[I 2021-12-15 17:14:30,263] Trial 434 finished with value: 0.8831211230819458 and parameters: {'n_estimators': 1000, 'max_depth': 11}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:14:42,000] Trial 435 finished with value: 0.8867123734900424 and parameters: {'n_estimators': 500, 'max_depth': 6}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:15:00,852] Trial 436 finished with value: 0.8792033953640223 and parameters: {'n_estimators': 500, 'max_depth': 9}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:15:35,926] Trial 437 finished with value: 0.8854064642507345 and parameters: {'n_estimators': 500, 'max_depth': 11}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:15:58,463] Trial 438 finished with value: 0.8792033953640223 and parameters: {'n_estimators': 500, 'max_depth': 9}. Best is trial 66 with value: 0.8867123734900424.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: Us

[I 2021-12-15 17:24:41,762] Trial 459 finished with value: 0.8788769180541952 and parameters: {'n_estimators': 1000, 'max_depth': 5}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:25:11,192] Trial 460 finished with value: 0.881815213842638 and parameters: {'n_estimators': 500, 'max_depth': 15}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:25:17,138] Trial 461 finished with value: 0.8824681684622919 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 66 with value: 0.8867123734900424.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:25:40,493] Trial 462 finished with value: 0.8788769180541952 and parameters: {'n_e

[I 2021-12-15 17:35:36,042] Trial 491 finished with value: 0.881815213842638 and parameters: {'n_estimators': 500, 'max_depth': 15}. Best is trial 66 with value: 0.8867123734900424.
C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\optuna\samplers\_grid.py:173: UserWarning: The value `1000` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntUniformDistribution(high=500, low=100, step=1)`.
  f"The value `{param_value}` is out of range of the parameter `{param_name}`. "
[I 2021-12-15 17:36:35,972] Trial 492 finished with value: 0.881162259222984 and parameters: {'n_estimators': 1000, 'max_depth': 6}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:36:50,645] Trial 493 finished with value: 0.8824681684622919 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:37:16,488] Trial 494 finished with value: 0.881162259222984 and parameters: {'n_est

[I 2021-12-15 17:54:52,087] Trial 516 finished with value: 0.881815213842638 and parameters: {'n_estimators': 1000, 'max_depth': 7}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:54:57,175] Trial 517 finished with value: 0.8847535096310807 and parameters: {'n_estimators': 100, 'max_depth': 6}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:55:09,360] Trial 518 finished with value: 0.8824681684622919 and parameters: {'n_estimators': 100, 'max_depth': 13}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:55:25,001] Trial 519 finished with value: 0.8824681684622919 and parameters: {'n_estimators': 100, 'max_depth': 17}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:56:07,974] Trial 520 finished with value: 0.8792033953640223 and parameters: {'n_estimators': 500, 'max_depth': 9}. Best is trial 66 with value: 0.8867123734900424.
[I 2021-12-15 17:57:13,040] Trial 521 finished with value: 0.8824681684622919 and parame

KeyboardInterrupt: 

In [146]:
tstamp1 = cmaes_study.trials_dataframe()['datetime_complete'].min()
tstamp2 = grid_study.trials_dataframe()['datetime_complete'].max()

In [147]:
temp = tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1
temp

Timedelta('0 days 00:02:46.584825')

In [148]:
print('GRID SEARCH: ',study.best_value)
print('CMAES: ',cmaes_study.best_value)
print('TPE: ',tpe_study.best_value)
print('RANDOM SEARCH : ',random_study.best_value)
print('BASELINE MODEL: ',accuracy)

GRID SEARCH:  0.8867123734900424
CMAES:  0.8821416911524649
TPE:  0.8896506692784851
RANDOM SEARCH :  0.8880182827293504
BASELINE MODEL:  0.8854064642507345


In [150]:
tstamp1 = tpe_study.trials_dataframe()['datetime_complete'].min()
tstamp2 = tpe_study.trials_dataframe()['datetime_complete'].max()
temp = tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1
temp

Timedelta('0 days 00:11:34.798708')